# Laboratório Elastic Search

# Atividade

## Contexto
O objetivo deste laboratório é explorar diferentes mecanismos de busca dentro e fora do Elastic Search!
Para isto, iremos explorar uma base de verificações de notícia de uma agência de checagem, chamada [Lupa](https://lupa.uol.com.br/).
O papel de uma agência de checagem é analisar uma notícia e verificar sua veracidade, gerar um veredito e retornar sua análise.

Neste laboratório, iremos nos preocupar apenas com o texto da análise e em como recuperá-los através de diferentes estratégias de busca.

## Tarefas
### Tarefa 1
- Cada uma das equipes receberá 2 queries fixas, chamadas de QF1 e QF2, e a tarefa do grupo será a criação de mais 2 queries, chamadas de QP1 e QP2. As queries devem seguir o formato de uma pergunta ou declaração textual e ela será usada na próxima tarefa para realizar buscas. As equipes podem se inspirar nas notícias fornecidas (presentes no csv da tarefa 2) para montá-las. Cada query deve ser composta por até 100 caracteres.
- A equipe deve enviar suas QP1 e QP2 em um formulário que será disponibilizado no classroom e no discord.

### Tarefa 2
- Após a criação das QP1 e QP2, cada equipe possuirá 4 queries (QF1; QF2; QP1 e QP2).
- Cada grupo irá realizar 4 tipos de busca. Uma busca léxica com BM25 (pelo ElasticSearch), uma busca semântica (pelo ElasticSearch), uma busca híbrida (manualmente utilizando as duas buscas anteriores) e uma estratégia de busca de sua preferência (neste lab ela será chamada de busca criativa), diferente das anteriores.
- As buscas léxica e semântica já estão implementadas, os grupos devem gerar a implementação das buscas híbridas e da busca criativa.
- A busca criativa será a busca usada para a competição!
- Cada grupo durante a implementação de suas buscas deve testar diferentes pré-processamentos nos dados para verificar como os resultados podem melhorar. Alguns pré-processamentos já foram implementados, mas os grupos não precisam se limitar aos pré-processamentos apresentados. Vocês podem buscar na internet implementações de outros pré-processamentos, criar padrões regex, etc. Observe que os dados precisarão ser reindexados no ElasticSearch sempre que os pré-processamentos forem modificados (inseridos, removidos ou mudados de ordem)!
- Enquanto vocês testam seus pré-processamentos e algoritmos de busca, ao analisar os resultados obtidos para uma determinada query, anotem o nível de relevância de cada documento lido em relação à query. Observe que esse procedimento pode ser realizado continuamente durante o processo de implementação, pois a relevância de um documento para uma query independe de implementação de buscadores. Essa anotação deve ser realizada (online) em uma planilha no google drive, que será fornecida pela organização do Lab. Caso mais de um integrante do grupo anote valores diferentes na planilha, não tem problemas. Nesse caso, iremos considerar a média dos valores.
    - A planilha tem o seguinte formato:
        - doc_id, query_id, relevance
        - 120, QP1, 2
        - 487, QP1, 1
        - ...
    - Cada busca deve possuir pelo menos 10 resultados anotados.
    - Não modifique o código que realiza o carregamento dos dados para que o doc_id seja consistente com os demais grupos!
    - A rotulação deve possui uma gradação em 3 níveis:
        - 0: Não é Relevante
        - 1: Pouco Relevante
        - 2: Muito Relevante 
- A entrega desta tarefa será feita através de um formulário para entregar um .zip contendo todo o repositório e o código utilizado autocontido (com listagem das dependências utilizadas no requirements.txt se necessário) e suas buscas implementadas. Este código será reexecutado, então organize o código para que ele possa ser reexecutado em outra máquina.
- Quando esgotar o prazo de envio da tarefa a planilha será bloqueada para modificações.
- A partir deste momento, nenhum grupo poderá alterar suas buscas novamente, então tenham ciência que esta será sua implementação final que será usada para a competição.

### Tarefa 3
- O grupo receberá (pelo discord) 3 queries adicionais (QA1; QA2 e QA3).
- A tarefa do grupo será a execução do MESMO CÓDIGO submetido na tarefa anterior com estas 3 queries e a anotação dos dados (a planilha de anotações será disponibilizada pelo discord).
- Nesta etapa, o grupo NÃO PODE FAZER ALTERAÇÕES nos pré-processamentos definidos ou nas implementações de suas buscas.
- Quando esgotar o prazo de envio da tarefa a planilha será bloqueada para modificações.
- IMPORTANTE: O seu código entregue na tarefa 2 será reexecutado com estas 3 queries (assim como você fez) para garantir que os resultados batem com os seus. Os grupos que mudarem suas buscas ou pré-processamentos na tarefa 3 terão sua nota penalizada e serão desclassificados da competição.

## Competição
A competição será realizada através da comparação dos resultados das buscas criativas em uma base de documentos (corpus) utilizada pela organização do Lab.
Para que você atinja um bom desempenho na competição, é importante se atentar ao seu processo de criação de queries e na qualidade da rotulação dos documentos, pois elas serão seus guias do quão boa sua busca está se saindo!

Como métrica de avaliação, esta competição irá utilizar a média do NDCG calculado para cada query. 

## Ferramental
Para executar este lab não é necessário uma máquina com GPU, mas a máquina deve ter capacidade de virtualização, além de possuir o Docker e o Python instalado.

Será utilizado o ElasticSearch (e opcionalmente o Kibana) no ambiente docker.

Para realizar o lab é recomendado o uso de Python 3.12, além de um venv ou ambiente conda.

Você deve instalar as dependências do requirements.txt.

## Detalhes
Ao executar o docker compose, além de ser levantado o ElasticSearch, também é levantada uma interface visual chamada Kibana.

O Kibana é um frontend para facilitar o acionamento de algumas operações do ElasticSearch e controle de configurações específicas, observar métricas etc. Ela pode ser utilizada para fins de debug. Para quem tiver curiosidade, acesse o URI: http://localhost:5601 após levantar o serviço com o docker compose.

## Dúvidas
Caso tenham dúvidas, é só entrar em contato pelo nosso servidor do Discord.

-----------------------
# Código


## Passos de preparação do ambiente

### Imports

In [1]:
import zipfile
import requests
import os
import pandas as pd
from datetime import datetime
import re
import warnings
import subprocess
warnings.filterwarnings("ignore")
from collections import OrderedDict

from elasticsearch import Elasticsearch

from sentence_transformers import SentenceTransformer

import nltk
from nltk import word_tokenize
from nltk.stem import PorterStemmer
import sklearn

nltk.download('punkt_tab')
nltk.download('stopwords')

import spacy
import unidecode


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/samuel.matos/Documents/studies/LabElasticSearch---Group8/.venv/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/samuel.matos/Documents/studies/LabElasticSearch---Group8/.venv/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/samuel.matos/Documents/studies/LabElast

RuntimeError: Failed to import transformers.models.auto.modeling_auto because of the following error (look up to see its traceback):
Failed to import transformers.generation.utils because of the following error (look up to see its traceback):
register_pytree_node() got an unexpected keyword argument 'flatten_with_keys_fn'

### Passo 0: Subir Stack do Elastic (ElasticSearch e Kibana)

In [3]:
!python -m spacy download pt_core_news_sm # Caso o comando não funcione, execute-o no terminal


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 189, in _run_module_as_main
  File "<frozen runpy>", line 148, in _get_module_details
  File "<frozen runpy>", line 112, in _get_module_details
  File "/Users/samuel.matos/Documents/studies/LabElasticSearch---Group8/.venv/lib/python3.12/site-packages/spacy/__init__.py", line 6, in <module>
    from .errors import setup_default_warnings
  File "/Users/samuel.matos/Documents/studies/LabElasticSearch---Group8/.venv/lib/python3.12/site-packages/spacy/errors.py", line 3, in <module>
    from .compa

In [5]:
subprocess.call(["docker", "compose", "up", "-d"])
# Se esse comando falhar ou retornar 1, execute-o diretamente no terminal para identificar o erro.
# PS: O docker deve estar instalado e rodando

time="2025-03-25T11:25:11-03:00" level=warning msg="/Users/samuel.matos/Documents/studies/LabElasticSearch---Group8/docker-compose.yaml: the attribute `version` is obsolete, it will be ignored, please remove it to avoid potential confusion"
 es01 Pulling 
 setup Pulling 
 kibana Pulling 
 4bb953b2341e Pulling fs layer 
 8e24285bc7e9 Pulling fs layer 
 712c04fdbd90 Pulling fs layer 
 4ca545ee6d5d Pulling fs layer 
 c8e302a2e0d1 Pulling fs layer 
 2920558bb9da Pulling fs layer 
 425da96d0239 Pulling fs layer 
 542cc10a95ab Pulling fs layer 
 a0a8cf8ea932 Pulling fs layer 
 425da96d0239 Waiting 
 ca2cb92388d3 Pulling fs layer 
 542cc10a95ab Waiting 
 a0a8cf8ea932 Waiting 
 ca2cb92388d3 Waiting 
 2920558bb9da Waiting 
 c8e302a2e0d1 Waiting 
 4bb953b2341e Pulling fs layer 
 0961e5ed6e0a Pulling fs layer 
 6ce04461e85c Pulling fs layer 
 f4f59eec995f Pulling fs layer 
 2ffd3541d0b7 Pulling fs layer 
 53b1f59a050f Pulling fs layer 
 4ca545ee6d5d Pulling fs layer 
 07997af583d7 Pulling fs laye

KeyboardInterrupt: 

### Passo 1: Baixar dados do Lupa

In [ ]:
# Base de dados de notícias da Lupa
url = "https://docs.google.com/uc?export=download&confirm=t&id=1W067Md2EbvVzW1ufzFg17Hf7Y9cCZxxr"
filename = "articles_lupa_lab_elasticsearch.zip"
data_path = "data"
zip_file_path = f"{data_path}/{filename}"

os.makedirs(data_path, exist_ok=True)

# Baixa o zip
with open(zip_file_path, "wb") as f:
    f.write(requests.get(url, allow_redirects=True).content)

# Extrai o csv do zip
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(data_path)
    
output_file = f"{data_path}/articles_lupa.csv"
assert os.path.exists(output_file)

### Passo 2: Pré-processar os dados e gerar embeddings

In [ ]:
# Implementações de pré-processamentos de texto. Modifiquem, adicionem, removam conforme necessário.
class Preprocessors:
    STOPWORDS = set(nltk.corpus.stopwords.words('portuguese'))
    
    def __init__(self, idf_limiar):
        self.stemmer = PorterStemmer()
        self.spacy_nlp = spacy.load("pt_core_news_sm") # Utiliza para lematização
        self.idf_limiar = idf_limiar
        
    # Remove stopwords do português
    def remove_stopwords(self, text):
        # Tokeniza as palavras
        tokens = word_tokenize(text)
        # Remove as stop words
        tokens = [word for word in tokens if word not in self.STOPWORDS]

        return ' '.join(tokens)
    
    # Realiza a lematização
    def lemma(self, text):
        return " ".join([token.lemma_ for token in self.spacy_nlp(text)])
    
    # Realiza a stemização
    def porter_stemmer(self, text):
        # Tokeniza as palavras
        tokens = word_tokenize(text)

        for index in range(len(tokens)):
            # Realiza a stemização
            stem_word = self.stemmer.stem(tokens[index])
            tokens[index] = stem_word

        return ' '.join(tokens)

    # Transforma o texto em lower case
    def lower_case(self, str):
        return str.lower()

    # Remove urls com regex
    def remove_urls(self, text):
        url_pattern = r'https?://\S+|www\.\S+'
        without_urls = re.sub(pattern=url_pattern, repl=' ', string=text)
        return without_urls

    # Remove números com regex
    def remove_numbers(self, text):
        number_pattern = r'\b(?!\d{4}\b|\b[12]?\d{1,2}\b|\b[1-2]?\d{3}\b)\d+\b'
        without_number = re.sub(pattern=number_pattern,
    repl=" ", string=text)
        return without_number

    # Converte caracteres acentuados para sua versão ASCII
    def accented_to_ascii(self, text):
        text = unidecode.unidecode(text)
        return text
    
    # Remove palavras com IDF alto
    def remove_commonwords(self, text):
        tokens = [word for word in text if nltk.text.TextCollection.idf(word) >= self.idf_limiar]

        return ' '.join(tokens)

NameError: name 'nltk' is not defined

In [ ]:
# Carregar o modelo gerador de embeddings
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Caminho para salvar o dataframe de notícias
data_df_path = "data/data_df.pkl"

# Selecione diferentes pré-processamentos
# Exemplo:
"""
preprocessor = Preprocessors()
preprocessing_steps = [
    preprocessor.remove_urls,
    preprocessor.remove_stopwords,
]
"""
preprocessor = Preprocessors(20)
preprocessing_steps = [
    preprocessor.lower_case,
    preprocessor.remove_urls,
    preprocessor.accented_to_ascii,
    preprocessor.remove_numbers,
    preprocessor.remove_stopwords,
    preprocessor.porter_stemmer
]

titlepreprocessing_steps = [
    preprocessor.lower_case,
    preprocessor.accented_to_ascii,
    preprocessor.remove_stopwords,
    preprocessor.remove_commonwords,
    preprocessor.remove_numbers,
    preprocessor.lemma
]

RECREATE_DF = True

# Cria o data frame se ele já existir ou se a variável RECREATE_INDEX for verdadeira
# Ou (exclusivo) carrega o dataframe salvo
if not os.path.exists(data_df_path) or RECREATE_DF:    
    df = pd.read_csv(output_file, sep=";")[["Título", "Texto", "Data de Publicação"]]
    df["Data de Publicação"] = df["Data de Publicação"].apply(lambda str_date: datetime.strptime(str_date.split(" - ")[0], "%d.%m.%Y"))
    df.sort_values("Data de Publicação", inplace=True, ascending=False)
    df["Embeddings"] = [None] * len(df)
    df["doc_id"] = df.reset_index(drop=True).index

    for i, row in df.iterrows():
        texto_completo = row["Texto"].strip() + "\n" + row["Título"].strip()
        
        df.at[i, "Texto completo"] = texto_completo
        texto_processado = texto_completo
        for preprocessing_step in preprocessing_steps:
            texto_processado = preprocessing_step(texto_processado)
        
        titulo = row["Título"].strip()
        titulo_processado = titulo
        for preprocessing_step in titlepreprocessing_steps:
            titulo_processado = preprocessing_step(titulo_processado)
        
        df.at[i, "Texto processado"] = texto_processado
        embeddings = model.encode(texto_completo).tolist()
        df.at[i, "Embeddings"] = embeddings
        
        df.at[i, "Titulo processado"] = titulo_processado
        title_embeddings = model.encode(titulo_processado).tolist()
        df.at[i, "TitleEmbeddings"] = title_embeddings
        
    print("Geração de embeddings finalizada.")
    
    with open(data_df_path, "wb") as f:
        df.to_pickle(f)
else:
    with open(data_df_path, "rb") as f:
        df = pd.read_pickle(f)
    print("Dataframe carregado de arquivo.")

Geração de embeddings finalizada.


### Passo 3: Indexar dados no ElasticSearch (Lembrem-se de reindexar os dados se os pré-processamentos mudarem)

In [ ]:
es = Elasticsearch(
    hosts = [{'host': "localhost", 'port': 9200, "scheme": "https"}],
    basic_auth=("elastic","elastic"),
    verify_certs = False,
)

In [ ]:
RECREATE_INDEX = True

index_name = "verificacoes_lupa"

# Se a flag for True e se o índice existir, ele é deletado
if RECREATE_INDEX and es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)
    print(f"Índice '{index_name}' deletado.")

# Cria o índice e popula com os dados
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name, mappings={
        "properties": {
            "doc_id": {"type": "integer"},
            "full_text": {"type": "text"},
            "processed_text": {"type": "text"},
            "processed_title": {"type": "text"},
            "embeddings": {"type": "dense_vector", "dims": 384},
            "title_embeddings": {"type": "dense_vector", "dims": 384}
        }
    })
    print(f"Índice '{index_name}' criado.")
    
    for i, row in df.iterrows():
        es.index(index=index_name, id=row["doc_id"], body={
            "doc_id": row["doc_id"],
            "full_text": row["Texto completo"],
            "processed_text": row["Texto processado"],
            "processed_title": row["Titulo processado"],
            "embeddings": row["Embeddings"],
            "title_embeddings": row["TitleEmbeddings"]
        })
    print("Índice preenchido.")

print("Indexação finalizada.")

Índice 'verificacoes_lupa' deletado.
Índice 'verificacoes_lupa' criado.
Índice preenchido.
Indexação finalizada.


## Tarefas

### Tarefa 1: Criar query
Crie as queries da tarefa 1 (QP1 e QP2), as queries devem ser perguntas ou declarações de até 100 caracteres.

Inspire-se nas notícias presente no csv (data/articles_lupa.csv) para gerar queries interessantes.

Submetam as queries ao formulário.

### Tarefa 2: Implementação e realização das buscas
Agora que você montou suas queries, realize cada uma das buscas para cada uma das queries (QF1; QF2; QP1 e QP2).

In [ ]:
# Estas serão as queries QF1 e QF2
with open("data/queries_fixadas.txt", "r") as f:
    queries_fixadas = [line.strip() for line in f.readlines()]
    assert len(queries_fixadas) == 2
    QF1 = queries_fixadas[0]
    QF2 = queries_fixadas[1]
    
# Preencha aqui as queries do grupo
QP1 = "Como se proteger de novos golpes online"
QP2 = "Existem leis anti discriminação e violência política no Brasil?"

queries = OrderedDict()
queries["QF1"] = QF1
queries["QF2"] = QF2
queries["QP1"] = QP1
queries["QP2"] = QP2

#### Busca Léxica

In [ ]:
# Implementação de busca esparsa (léxica) com BM25
def lexical_search(queries: dict[str, str]):
    lexical_results = {}
    for query_id, query in queries.items():
        
        # Pré-processa os dados
        for preprocessing_step in preprocessing_steps:
            query = preprocessing_step(query)
        
        search_query = {
            "size": 60,
            "query": {
                "match": {
                    "processed_text": query
                }
            }
        }

        # Realiza a busca
        response = es.search(index=index_name, body=search_query)
        
        hits_results = []
        # Recupera os resultados
        for hit in response["hits"]["hits"]:
            hits_results.append((hit["_source"]["doc_id"], hit["_score"]))
        lexical_results[query_id] = hits_results
        
    return lexical_results

#### Busca Semântica

In [ ]:
# Realiza busca semântica (densa) com KNN exato
def semantic_search(queries: dict[str, str]):
    semantic_results = {}
    
    for query_id, query in queries.items():
        # Aplica todos os pré-processamentos aos dados
        for preprocessing_step in preprocessing_steps:
            query = preprocessing_step(query)
            
        query_vector = model.encode(query).tolist()
        
        
        search_query = {
            "size": 60,
            "query": {
                "script_score": {
                    "query": {"match_all": {}},
                    "script": {
                        "source": "cosineSimilarity(params.query_vector, 'embeddings') + 1.0",
                        "params": {"query_vector": query_vector}
                    }
                }
            }
        }

        # Realiza a busca
        response = es.search(index=index_name, body=search_query)
        
        hits_results = []
        # Recupera top 10 resultados
        for hit in response["hits"]["hits"]:
            hits_results.append((hit["_source"]["doc_id"], hit["_score"]))
            
        semantic_results[query_id] = hits_results

    return semantic_results

#### Busca Híbrida

In [ ]:
# Busca híbrida ou RRF. Implemente sua solução aqui. Você pode realizar as duas buscas anteriores (léxica e semântica) como base para formar a busca híbrida.
def hybrid_search(queries: dict[str, str]):
    lexical_results = lexical_search(queries)
    restricted_semantic_results = {}

    for query_id, query in queries.items():
        # Aplica pré-processamentos
        for preprocessing_step in preprocessing_steps:
            query = preprocessing_step(query)
        
        # Codifica query em vetor denso
        query_vector = model.encode(query).tolist()

        # Obtém os doc_ids da busca léxica
        lexical_doc_ids = [doc_id for doc_id, _ in lexical_results.get(query_id, [])]
        
        if not lexical_doc_ids:
            restricted_semantic_results[query_id] = []
            continue

        # Monta query semântica restrita aos doc_ids
        search_query = {
            "size": 60,
            "query": {
                "script_score": {
                    "query": {
                        "terms": {
                            "doc_id": lexical_doc_ids
                        }
                    },
                    "script": {
                        "source": "cosineSimilarity(params.query_vector, 'embeddings') + 1.0",
                        "params": {"query_vector": query_vector}
                    }
                }
            }
        }

        response = es.search(index=index_name, body=search_query)
        
        hits_results = []
        for hit in response["hits"]["hits"]:
            hits_results.append((hit["_source"]["doc_id"], hit["_score"]))
        
        restricted_semantic_results[query_id] = hits_results

    return restricted_semantic_results

#### Busca Criativa

In [ ]:
# Implemente sua própria estratégia de busca, podendo ela ser esparsa, densa ou híbrida. Implemente algo como "more_like_this", "BM35", "fuzzy" etc.
def creative_search(queries: dict[str, str]):
    creative_results = {}

    for query_id, query in queries.items():
        # Aplica todos os pré-processamentos aos dados
        for preprocessing_step in preprocessing_steps:
            query = preprocessing_step(query)
            
        query_vector = model.encode(query).tolist()
        
        search_query = {
            "size": 60,
            "query": {
                "script_score": {
                    "query": {"match_all": {}},
                    "script": {
                        "source": "cosineSimilarity(params.query_vector, 'title_embeddings') + 1.0",
                        "params": {"query_vector": query_vector}
                    }
                }
            }
        }

        # Realiza a busca
        response = es.search(index=index_name, body=search_query)
        
        for hit in response["hits"]:
            hits_results.append((hit["_source"]["doc_id"], hit["_score"]))
            
        creative_results[query_id] = hits_results
        print(hits_results)
    # return creative_results

#### Execução das buscas

In [ ]:
search_functions = [
    ("lexical", lexical_search),
    ("semantic", semantic_search),
    ("hybrid", hybrid_search),
    #("creative", creative_search)
]

def run_all_searches(queries: dict[str, str]):
    all_search_results = {}
    for search_name, search_function in search_functions:
        results = search_function(queries)
        all_search_results[search_name] = results
    return all_search_results

#### Analise os resultados da busca e aprimore a busca!

In [ ]:
all_search_results = run_all_searches(queries)
search_results_df = pd.DataFrame(all_search_results)
search_results_df

,lexical,semantic
QF1,"[(1606, 22.101274), (1491, 17.99196), (1920, 1...","[(1920, 1.5544899), (2130, 1.537176), (307, 1...."


In [ ]:
def generate_exploded_df(search_results_df):
    exploded_search_results_df = pd.concat([search_results_df[col].explode() for col in search_results_df.columns], axis=1)
    exploded_search_results_df = exploded_search_results_df.apply(lambda l: [doc_id for doc_id, _ in l])
    return exploded_search_results_df

def generate_found_docs_text_df(exploded_search_results_df, all_docs_df):
    # Recupera os ids únicos dos documentos
    documents_ids = set(exploded_search_results_df.to_numpy().flatten().tolist())

    # Salva os textos e os ids dos documetnos que foram encontrados ems usas buscas
    documents_df = all_docs_df[all_docs_df["doc_id"].isin(documents_ids)][["Texto processado", "doc_id", "Texto completo"]]
    return documents_df

exploded_df = generate_exploded_df(search_results_df)
found_docs_text_df = generate_found_docs_text_df(exploded_df, all_docs_df=df)

def save_results_to_file(exploded_df: pd.DataFrame,
                         found_docs_text_df: pd.DataFrame,
                         exploded_df_save_filepath: str = "data/search_results.csv",
                         found_docs_text_save_filepath: str = "data/documents.csv"):
    exploded_df.to_csv(exploded_df_save_filepath)
    found_docs_text_df.to_csv(found_docs_text_save_filepath)
    print("Resultados das buscas salvos em 'data/search_results.csv'.")
    print("Documentos de interesse salvos em 'data/documents.csv'.")
    
save_results_to_file(exploded_df, found_docs_text_df)
#exploded_df

Resultados das buscas salvos em 'data/search_results.csv'.
Documentos de interesse salvos em 'data/documents.csv'.


,lexical,semantic
QF1,1606,1920
QF1,1491,2130
QF1,1920,307
QF1,88,1606
QF1,1660,2255
QF1,623,1087
QF1,2650,1991
QF1,1766,1660
QF1,4090,1491
QF1,1582,1634


In [ ]:
os.makedirs('output', exist_ok=True)
for query in queries:
    os.makedirs('output/' + query, exist_ok=True)
    for col in exploded_df:
        rel_df = {'DocID' : [], 'Título': [], 'Texto completo': []}
        for doc_id in exploded_df[col]:
            doc = df[df['doc_id'] == doc_id][['doc_id', 'Título', 'Texto completo']]
            idx = int(doc['doc_id'].index[0])
            rel_df['DocID'].append(doc_id)
            rel_df['Título'].append(doc['Título'][idx])
            rel_df['Texto completo'].append(doc['Texto completo'][idx])
        
        pd.DataFrame(rel_df).to_csv('output/' + query + '/' + col + '.csv', index=False)

#### Anote as relevâncias na sua planilha!

### Tarefa 3: Reexecutar a busca para as novas queries e rotular os dados

In [ ]:
# Preencha aqui com as queries adicionais do seu grupo
QA1 = XXXXXXXXX
QA2 = XXXXXXXXX
QA3 = XXXXXXXXX

queries = OrderedDict()
queries["QF1"] = QF1
queries["QF2"] = QF2
queries["QA1"] = QA1
queries["QA2"] = QA2
queries["QA3"] = QA3


all_search_results = run_all_searches(queries)
search_results_df = pd.DataFrame(all_search_results)
search_results_df

In [ ]:
exploded_df = generate_exploded_df(search_results_df)
found_docs_text_df = generate_found_docs_text_df(exploded_df, all_docs_df=df)

def save_results_to_file(exploded_df: pd.DataFrame,
                         found_docs_text_df: pd.DataFrame,
                         exploded_df_save_filepath: str = "data/search_results.csv",
                         found_docs_text_save_filepath: str = "data/documents.csv"):
    exploded_df.to_csv(exploded_df_save_filepath)
    found_docs_text_df.to_csv(found_docs_text_save_filepath)
    print("Resultados das buscas salvos em 'data/search_results.csv'.")
    print("Documentos de interesse salvos em 'data/documents.csv'.")
    
save_results_to_file(exploded_df, found_docs_text_df)
exploded_df

#### Anote os resultados na sua planilha!

#### A competição utilizará o NDCG médio por query para computar seu desempenho.